In [2]:
from transformers import pipeline

# Dùng đúng BERT-base
mask_filler = pipeline(
    "fill-mask",
    model="bert-base-uncased",
    tokenizer="bert-base-uncased"
)

input_sentence = "Hanoi is the [MASK] of Vietnam."

predictions = mask_filler(input_sentence, top_k=5)

print(f"Câu gốc: {input_sentence}\n")
for pred in predictions:
    print(f"Dự đoán: '{pred['token_str']}' với độ tin cậy: {pred['score']:.4f}")
    print(f" -> Câu hoàn chỉnh: {pred['sequence']}\n")


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


Câu gốc: Hanoi is the [MASK] of Vietnam.

Dự đoán: 'capital' với độ tin cậy: 0.9991
 -> Câu hoàn chỉnh: hanoi is the capital of vietnam.

Dự đoán: 'center' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the center of vietnam.

Dự đoán: 'birthplace' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the birthplace of vietnam.

Dự đoán: 'headquarters' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the headquarters of vietnam.

Dự đoán: 'city' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the city of vietnam.



Bài 1 – Masked Language Modeling với BERT

Câu 1 – Mô hình đã dự đoán đúng từ capital không?

Kết quả thực tế:

Câu gốc: Hanoi is the [MASK] of Vietnam.

Dự đoán top-1: 'capital' với độ tin cậy: ~0.9991

Câu hoàn chỉnh: hanoi is the capital of vietnam.

Các từ còn lại trong top-5: center, birthplace, headquarters, city có score rất nhỏ.

 Kết luận: Mô hình đã dự đoán đúng từ cần điền là capital, phù hợp với nghĩa câu “Hanoi is the capital of Vietnam.”

 Câu 2 – Tại sao mô hình Encoder-only như BERT phù hợp cho tác vụ này?

BERT là mô hình encoder-only, bidirectional: khi đoán [MASK], nó nhìn được cả ngữ cảnh trước và sau:

Trước: “Hanoi is the …”

Sau: “… of Vietnam.”

Mô hình được huấn luyện trực tiếp với mục tiêu Masked Language Modeling (MLM): che một số token, bắt mô hình dự đoán lại dựa trên toàn bộ ngữ cảnh hai chiều.

Vì thế, BERT rất phù hợp với bài toán điền từ vào chỗ trống (masked token), như ví dụ “Hanoi is the [MASK] of Vietnam”.

(Trong khi đó, mô hình decoder-only như GPT chỉ nhìn được quá khứ nên không thích hợp cho MLM kiểu này.)

In [3]:
from transformers import pipeline

# 1. Tải pipeline "text-generation"
generator = pipeline("text-generation")  # mặc định thường là gpt-2

# 2. Đoạn văn bản mồi
prompt = "The best thing about learning NLP is"

# 3. Sinh văn bản
generated_texts = generator(
    prompt,
    max_length=50,          # tổng độ dài câu mồi + phần sinh ra
    num_return_sequences=1, # trả về 1 câu
    do_sample=True,         # sampling để câu đa dạng hơn (có thể bỏ)
    top_k=50,               # optional
    top_p=0.95              # optional
)

# 4. In kết quả
print(f"Câu mồi: '{prompt}'\n")
for text in generated_texts:
    print("Văn bản được sinh ra:")
    print(text["generated_text"])
    print()


No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Câu mồi: 'The best thing about learning NLP is'

Văn bản được sinh ra:
The best thing about learning NLP is the fact that you can actually learn from it. You can learn from it because it's so easy to learn. And it's also so easy to learn because you don't have to understand anything. You can learn from it because you can practice. You can learn from it because you can be in a certain state and learn something.

So there's a lot of things you can learn from NLP. So you can learn from it.

So it's a really important thing to have and I think it's really important to have in your life because I think it's really important to have this great sense of being able to make sense of everything. I think that's really important to have. I think that's really important to have. I think that's really important to have. And I think it's really important to have. And I think that's really important to have. So, let me just say that it's really important to have. And I think it's really important to h

Bài 2 – Next Token Prediction với GPT (Text Generation)

Câu 1 – Kết quả sinh ra có hợp lý không?

Câu mồi:

The best thing about learning NLP is

Đoạn văn GPT-2 sinh ra (rút gọn):

“… the fact that you can actually learn from it. You can learn from it because it's so easy to learn. … So there's a lot of things you can learn from NLP. … it's really important to have in your life because I think it's really important to have this great sense of being able to make sense of everything. I think that's really important to have. I think that's really important to have. …”

Nhận xét:

Nội dung xoay quanh việc học NLP, học được nhiều điều, quan trọng, phù hợp với ngữ cảnh câu mồi → hợp lý về mặt chủ đề và ngữ nghĩa.

Câu tiếng Anh khá tự nhiên, tuy nhiên có hiện tượng lặp ý/lặp cụm “really important to have” nhiều lần, là hành vi thường thấy ở GPT-2.

 Kết luận: Kết quả sinh ra hợp lý, đúng chủ đề, dù còn lặp lại và hơi dài dòng.

Câu 2 – Tại sao mô hình Decoder-only như GPT phù hợp cho tác vụ này?

GPT là mô hình decoder-only, autoregressive, được huấn luyện với mục tiêu dự đoán token tiếp theo (Next Token Prediction / Causal LM) dựa trên tất cả token đứng trước.

Self-attention trong GPT dùng causal mask: tại bước thứ t, mô hình chỉ được nhìn các vị trí từ 1 đến t−1, không được nhìn “tương lai”. Điều này đúng với cách ta sinh văn bản từ trái sang phải.

Khi generate:

Bắt đầu từ prompt (“The best thing about learning NLP is”),

Mô hình dự đoán từ tiếp theo,

Ghép vào chuỗi, dùng làm input để dự đoán từ tiếp theo nữa, lặp lại cho tới khi đủ dài.

 Vì kiến trúc và cách huấn luyện đều được thiết kế cho việc sinh chuỗi liên tiếp, mô hình decoder-only như GPT rất phù hợp cho tác vụ text generation / next token prediction.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

# 1. Chọn mô hình BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Câu đầu vào
sentences = ["This is a sample sentence."]

# 3. Tokenize
inputs = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# 4. Đưa qua mô hình để lấy hidden states
with torch.no_grad():
    outputs = model(**inputs)

# outputs.last_hidden_state: (batch_size, sequence_length, hidden_size)
last_hidden_state = outputs.last_hidden_state

# 5. Mean Pooling (bỏ qua token padding)
attention_mask = inputs["attention_mask"]                 # (B, T)
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

sum_embeddings = torch.sum(last_hidden_state * mask_expanded, dim=1)  # (B, H)
sum_mask = torch.clamp(mask_expanded.sum(dim=1), min=1e-9)            # (B, 1)

sentence_embedding = sum_embeddings / sum_mask  # (B, H)

# 6. In kết quả
print("Vector biểu diễn của câu:")
print(sentence_embedding)
print("\nKích thước của vector:", sentence_embedding.shape)


Vector biểu diễn của câu:
tensor([[-6.3874e-02, -4.2837e-01, -6.6779e-02, -3.8430e-01, -6.5784e-02,
         -2.1826e-01,  4.7636e-01,  4.8659e-01,  4.0647e-05, -7.4273e-02,
         -7.4740e-02, -4.7635e-01, -1.9773e-01,  2.4824e-01, -1.2162e-01,
          1.6678e-01,  2.1045e-01, -1.4576e-01,  1.2636e-01,  1.8635e-02,
          2.4640e-01,  5.7090e-01, -4.7014e-01,  1.3782e-01,  7.3650e-01,
         -3.3808e-01, -5.0331e-02, -1.6452e-01, -4.3517e-01, -1.2900e-01,
          1.6516e-01,  3.4004e-01, -1.4930e-01,  2.2422e-02, -1.0488e-01,
         -5.1916e-01,  3.2964e-01, -2.2162e-01, -3.4206e-01,  1.1993e-01,
         -7.0148e-01, -2.3126e-01,  1.1224e-01,  1.2550e-01, -2.5191e-01,
         -4.6374e-01, -2.7261e-02, -2.8415e-01, -9.9249e-02, -3.7017e-02,
         -8.9192e-01,  2.5005e-01,  1.5816e-01,  2.2701e-01, -2.8497e-01,
          4.5300e-01,  5.0945e-03, -7.9441e-01, -3.1008e-01, -1.7403e-01,
          4.3029e-01,  1.6816e-01,  1.0590e-01, -4.8987e-01,  3.1856e-01,
          3.

Bài 3 – Sentence Embedding với BERT (Mean Pooling)

Câu 1 – Kích thước vector biểu diễn là bao nhiêu? Tương ứng tham số nào?

Với câu:

This is a sample sentence.

Kết quả in ra:

sentence_embedding.shape = torch.Size([1, 768])


1: batch size (1 câu).

768: số chiều của vector biểu diễn câu.

 Con số 768 chính là tham số hidden_size của mô hình bert-base-uncased, cũng là số chiều hidden state cho mỗi token trong last_hidden_state.

Câu 2 – Tại sao cần dùng attention_mask khi Mean Pooling?

Khi xử lý nhiều câu cùng lúc, tokenizer sẽ padding các câu ngắn bằng token <PAD> cho đủ độ dài.

last_hidden_state chứa vector của cả token thật lẫn token padding.

Nếu lấy trung bình (mean) tất cả vị trí (kể cả <PAD>), thì:

Các vector “rác” tại vị trí padding sẽ làm sai lệch và pha loãng sentence embedding.

attention_mask có giá trị:

1 tại vị trí token thật,

0 tại vị trí padding.

Khi nhân last_hidden_state * mask_expanded:

Các vị trí padding bị nhân với 0 → không đóng góp vào tổng.

Khi chia cho sum_mask, ta chia theo số token thật, không tính padding.

 Vì vậy, sử dụng attention_mask đảm bảo rằng vector biểu diễn câu chỉ được tính từ các token thực sự, không bị ảnh hưởng bởi các token <PAD>.